# 09 - Otimizadores

Neste notebook, vamos explorar diferentes algoritmos de otimização usados em aprendizado de máquina e deep learning. Começaremos com visualizações de como os otimizadores navegam por superfícies de funções e depois compararemos seu desempenho em um subconjunto do MNIST.

## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Fundamentos

Os algoritmos de otimização são essenciais para treinar redes neurais. Eles buscam minimizar uma função de custo $L(\theta)$ ajustando os parâmetros $\theta$ do modelo.

### Gradiente Descendente Básico

O algoritmo mais simples atualiza os parâmetros na direção oposta ao gradiente:

$$\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$$

onde $\eta$ é a taxa de aprendizado.

### Momentum

O momentum adiciona uma "memória" das atualizações anteriores:

$$v_{t+1} = \beta v_t + \nabla L(\theta_t)$$
$$\theta_{t+1} = \theta_t - \eta v_{t+1}$$

onde $\beta$ é o coeficiente de momentum (tipicamente 0.9).

### Visualizando steps

In [ ]:
def f(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def f_gradient(x, y):
    dx = 4*x*(x**2 + y - 11) + 2*(x + y**2 - 7)
    dy = 2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)
    return np.array([dx, dy])

# Função para simular otimização com PyTorch optimizers
def run_pytorch_optimizer(optimizer_class, optimizer_kwargs, start_point, n_steps=50):
    # Criando parâmetro como tensor
    position = torch.tensor(start_point, dtype=torch.float32, requires_grad=True)
    optimizer = optimizer_class([position], **optimizer_kwargs)
    
    steps = [position.detach().numpy().copy()]
    
    for _ in range(n_steps):
        optimizer.zero_grad()
        
        # Calculando loss
        x, y = position[0], position[1]
        loss = (x**2 + y - 11)**2 + (x + y**2 - 7)**2
        
        # Backward pass
        loss.backward()
        
        # Optimizer step
        optimizer.step()
        
        steps.append(position.detach().numpy().copy())
        
        # Stop se convergiu
        if torch.norm(position.grad) < 1e-3:
            break
    
    return steps

# Função para plotar contornos e passos do otimizador
def plot_contours_and_steps(X, Y, Z, steps_list, labels, title):
    plt.figure(figsize=(12, 8))
    
    # Plot contornos
    levels = np.logspace(0, 3, 20)
    cp = plt.contour(X, Y, Z, levels=levels, alpha=0.6)
    plt.contourf(X, Y, Z, levels=levels, alpha=0.3, cmap='viridis')
    
    # Plot passos dos otimizadores
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    for i, (steps, label) in enumerate(zip(steps_list, labels)):
        steps = np.array(steps)
        plt.plot(steps[:, 0], steps[:, 1], 'o-', 
                color=colors[i % len(colors)], 
                label=label, linewidth=2, markersize=6)
        plt.plot(steps[0, 0], steps[0, 1], 's', 
                color=colors[i % len(colors)], markersize=10, alpha=0.7)
    
    plt.title(title, fontsize=14)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('y', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Criando a superfície da função
x = np.linspace(-3.25, -2.4, 50)
y = np.linspace(2.5, 3.5, 50)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# Ponto inicial
start_point = [-3.0, 3.0]

# Testando diferentes valores de momentum usando torch.optim.SGD
momentum_values = [0.0, 0.3, 0.6, 0.9]
steps_list = []
labels = []

for momentum in momentum_values:
    steps = run_pytorch_optimizer(
        optim.SGD, 
        {'lr': 0.01, 'momentum': momentum}, 
        start_point, 
        n_steps=100
    )
    steps_list.append(steps)
    labels.append(f'Momentum = {momentum}')

plot_contours_and_steps(X, Y, Z, steps_list, labels, 'Comparação de SGD com Diferentes Valores de Momentum (PyTorch)')

## Otimizadores com MNIST

In [ ]:
# Carregando e preparando o dataset MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Carregando datasets completos
full_train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
full_test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

# Criando subconjuntos menores para comparação rápida
train_indices = torch.randperm(len(full_train_dataset))[:5000]
val_indices = torch.randperm(len(full_test_dataset))[:1000]

train_subset = torch.utils.data.Subset(full_train_dataset, train_indices)
val_subset = torch.utils.data.Subset(full_test_dataset, val_indices)

# DataLoaders
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_subset, batch_size=64, shuffle=False)

print(f"Tamanho do conjunto de treino: {len(train_subset)}\")\nprint(f\"Tamanho do conjunto de validação: {len(val_subset)}")

In [ ]:
# Modelo simples para comparação
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Função de treino
def train_model(model, train_loader, val_loader, optimizer, epochs=10):
    criterion = nn.CrossEntropyLoss()
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        # Treino
        model.train()
        epoch_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_loss)
        
        # Validação
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in val_loader:
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)
        
        if epoch % 2 == 0:
            print(f'Epoch {epoch}: Loss = {avg_loss:.4f}, Val Acc = {accuracy:.2f}%')

    return train_losses, val_accuracies

### Otimizadores Avançados

Antes de treinar, vamos entender os algoritmos que compararemos:

**AdaGrad**: Adapta a taxa de aprendizado baseada no histórico de gradientes
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} \nabla L(\theta_t)$$

**RMSprop**: Usa média móvel exponencial dos gradientes ao quadrado
$$v_t = \beta v_{t-1} + (1-\beta) (\nabla L(\theta_t))^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{v_t + \epsilon}} \nabla L(\theta_t)$$

**Adam**: Combina momentum com adaptação de taxa de aprendizado
$$m_t = \beta_1 m_{t-1} + (1-\beta_1) \nabla L(\theta_t)$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla L(\theta_t))^2$$

In [ ]:
# Configuração dos otimizadores
optimizers_config = {
    'SGD': {'lr': 0.01},
    'SGD+Momentum': {'lr': 0.01, 'momentum': 0.9},
    'Adagrad': {'lr': 0.01},
    'RMSprop': {'lr': 0.001},
    'Adam': {'lr': 0.001}
}

# Dicionário para armazenar resultados
results = {}

print("Comparando otimizadores no MNIST...")
print("="*50)

# Treinando com cada otimizador
for opt_name, config in optimizers_config.items():
    print(f"\nTreinando com {opt_name}...")

    # Criando novo modelo para cada otimizador
    model = SimpleNet()

    # Mapeamento de nomes para classes
    opt_map = {
        'SGD': optim.SGD,
        'SGD+Momentum': optim.SGD,
        'Adagrad': optim.Adagrad,
        'RMSprop': optim.RMSprop,
        'Adam': optim.Adam
    }
    
    # Instanciando otimizador
    optimizer = opt_map[opt_name](model.parameters(), **config)

    # Treinando
    train_losses, val_accuracies = train_model(
        model, train_loader, val_loader, optimizer, epochs=15
    )

    # Salvando resultados
    results[opt_name] = {
        'train_losses': train_losses,
        'val_accuracies': val_accuracies,
        'final_accuracy': val_accuracies[-1]
    }

    print(f"Acurácia final: {val_accuracies[-1]:.2f}%")

In [ ]:
# Visualizando os resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Curvas de loss de treino
for opt_name, result in results.items():
    ax1.plot(result['train_losses'], label=opt_name, linewidth=2)

ax1.set_xlabel('Época')
ax1.set_ylabel('Loss de Treino')
ax1.set_title('Comparação das Curvas de Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)
# ax1.set_ylim(0, max_loss)  # opcional: fixar limite superior

# Plot 2: Curvas de acurácia de validação
for opt_name, result in results.items():
    ax2.plot(result['val_accuracies'], label=opt_name, linewidth=2)

ax2.set_xlabel('Época')
ax2.set_ylabel('Acurácia de Validação (%)')
ax2.set_title('Comparação das Curvas de Acurácia')
ax2.legend()
ax2.grid(True, alpha=0.3)
# ax2.set_ylim(0, 100)  # opcional: manter escala de 0 a 100%

plt.tight_layout()
plt.show()

# Resumo dos resultados finais
print("\nResumo dos Resultados Finais:")
print("="*40)
for opt_name, result in results.items():
    print(f"{opt_name:<15} -> Acurácia Final: {result['final_accuracy']:.2f}%")